### Parte 3

In [7]:
import numpy as np
import pandas as pd
import plotly.express as px

#Parametros
D=50 #tamaño de la ciudad
N=200 #poblacion
r=0.6 #radio de infeccion
recovery_rate=0.05 #tasa de recuperacion
stdev=D/20 #desviacion estandar de la distribucion de la poblacion

#Problema 3- Cluster de ciudadanos
x0,y0=np.random.uniform(0,D),np.random.uniform(0,D) #posicion inicial del cluster de ciudadanos

#Posiciones iniciales de los ciudadanos
x_pos=np.clip(np.random.normal(x0,stdev,N),5,D-5)
y_pos=np.clip(np.random.normal(y0,stdev,N),5,D-5)

#estados de los ciudadanos
state=np.array(['suseptible']*(N))
state[np.random.choice(range(N),size=1)]=['infectado']
state[-1]='recuperado'
ids=np.arange(N) #id unico de ciudadanos

data=pd.DataFrame({'id':ids,'x':x_pos,'y':y_pos,'state':state,'iteracion':0})

def euclid_dist(x1, y1, x2, y2, r):
    distance = np.sqrt((x1 - x2)**2 + (y1 - y2)**2)
    if distance < r:
        return 1
    else:
        return 0



### Parte 4

In [8]:
iterations=50 #numero de iteraciones
frames=[data.copy()] #frames de la animacion


for t in range(1,iterations):
    #cambio de posicion
    x_dis=np.random.normal(0,stdev,N)
    y_dis=np.random.normal(0,stdev,N)

    data['x']+=x_dis
    data['y']+=y_dis
    data['x'] = np.clip(data['x'], 1, D-1)
    data['y'] = np.clip(data['y'], 1, D-1)
    
      
    #progreso de la infeccion
    for i,row in data.iterrows():
        if row['state']=='suseptible':
            for j,inf_row in data[data['state']=='infectado'].iterrows():
                if euclid_dist(row['x'],row['y'],inf_row['x'],inf_row['y'],r)==1:
                    data.at[i,'state']='infectado'
                    break
    
    #Recuperados de la infeccion
    for j, inf_row in data[data['state'] == 'infectado'].iterrows():
        r_prob = np.random.rand(1)
        if r_prob < recovery_rate:
            data.at[j, 'state'] = 'recuperado'
    
        
    #actualizar datos
    data['iteracion']=t
    frames.append(data.copy())

animacion=pd.concat(frames,ignore_index=True)

fig = px.scatter(
animacion,
x='x',
y='y',
color='state',
color_discrete_map={
        'suseptible': 'blue',
        'infectado': 'red',
        'recuperado': 'green'
    },
animation_frame='iteracion',
title='Simulacion Epidemica- Problema 3',
labels={'x': 'X', 'y': 'Y', 'state': 'Estado de Salud'},
category_orders={'state': ['suseptible', 'infectado', 'recuperado']},
range_x=[0, D],
range_y=[0, D],
width=700,
height=700
)

fig.update_traces(marker=dict(size=8))
fig.update_layout(legend=dict(title="State"))


# Display animation
fig.show()


